In [39]:
import json
import numpy as np
class Graph:
    """
    Class for creating file congruence graph.
    Nodes representing users. Edge exists between nodes if users got common changed files. Edge color and width depends
    on number of common files between users.
    """

    def __init__(self, M, mult=1, heading="Graph"):
        self.M = M
        self.user_to_id = load_json(f"{resources}/userToId")
        self.id_to_user = {v: k for k, v in self.user_to_id.items()}
        self.mult = mult

        self.__init_net(heading)

    def __gen_quantiles(self):
        """
        Generate quantiles (0.25, 0.5, 0.75) for non null values
        """
        non_null_values = self.M[self.M != 0]
        self.quantiles = [
            np.quantile(non_null_values, 0.25)
            , np.quantile(non_null_values, 0.5)
            , np.quantile(non_null_values, 0.75)
        ]
        print(self.quantiles)

    def __init_net(self, heading):
        """
        Create net-graph
        :param heading: heading
        """
        self.net = Network(heading=heading
                           , height="1080px"
                           , width="100%"
                           , bgcolor="#222222"
                           , font_color="white")

        self.__gen_quantiles()

        self.net.barnes_hut()

        for userId in range(M.shape[0]):
            for fileId in range(M.shape[1]):

                weight = self.M[userId, fileId] * self.mult
                if weight == 0:
                    continue
                src = self.id_to_user[userId]
                dst = str(fileId)

#                 print(f"{src} -> {dst} = {weight}")
                self.net.add_node(src, src, title=src)
                self.net.add_node(dst, dst, title=dst, color="red")
                self.net.add_edge(src, dst, value=weight, color=self.edge_color(self.M[userId, fileId], *self.quantiles))

#         neighbor_map = self.net.get_adj_list()

#         for node in self.net.nodes:
#             node["title"] += " Neighbors:<br>" + "<br>".join(self.__pretty_string_neighbors(node, neighbor_map))
#             node["value"] = len(neighbor_map[node["id"]])

    def __get_num_of_files_with_neighbor(self, user, neighbor):
        """
        Map function. Get number of common files with neighbor for user.
        :param user: user login
        :param neighbor: neighbor login
        :return: tuple (neighbor login, number of common files with neighbor)
        """
        num_of_files = self.M[self.user_to_id[user], self.user_to_id[neighbor]]
        return neighbor, num_of_files

    def __pretty_string_neighbors(self, node, neighbor_map):
        """
        Create string for better visualization.
        :param node: current node
        :param neighbor_map: neighbors for each node
        :return: string of neighbors and number of common files
        """
        result = map(lambda neighbor: self.__get_num_of_files_with_neighbor(node["title"], neighbor)
                     , neighbor_map[node["id"]])
        result = sorted(result, key=lambda x: x[1], reverse=True)
        result = map(lambda x: f"{x[0]} : {x[1]}", result)
        return result

    def show(self):
        print(f"{work_dir}/{self.net.heading}.html")
        self.net.show(f"{work_dir}/{self.net.heading}.html")


    @staticmethod
    def edge_color(weight, quantile1, quantile2, quantile3):
        if weight <= quantile1:
            return '#0569E1'
        if weight <= quantile2:
            return '#C1F823'
        if weight <= quantile3:
            return '#FCAA05'
        return '#EE5503'

In [2]:
def load_json(filename):
    with open(f"{filename}", 'r') as fp:
        data = json.load(fp)
    return data
def print_(M):
    print(f"min={M.min()}, mean={M.mean()}, max={M.max()}, shape={M.shape}")

In [12]:
resources = '/home/nikolaisv/study/intership/Analysis-of-Socio-Technical-Congruence/resources'
numFiles = len(load_json(f"{resources}/fileToId"))
numUsers = len(load_json(f"{resources}/idToUser"))
DK = load_json(f"{resources}/developerKnowledge")



In [17]:
DK.keys


<function dict.keys>

In [19]:
shape = (numUsers, numFiles)
M = np.zeros(shape)
for userId in DK.keys():
    for fileId in DK[userId].keys():
        M[int(userId)][int(fileId)] = DK[userId][fileId]

0.00011315164301947896

In [3]:
from pyvis.network import Network
work_dir = '/home/nikolaisv/study/intership/visualization/'

In [40]:
_M = np.copy(M)
_M[_M < 0.01] = 0
graph = Graph(_M, heading="developerKnowledge")
graph.show()

[0.020483518968303638, 0.054909025828380226, 0.2951991900837486]
/home/nikolaisv/study/intership/visualization//developerKnowledge.html
